In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hyperparameter optimization") \
  .getOrCreate()

24/10/12 22:28:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("data/hotel-reservations.csv", header=True, inferSchema=True)

root
 |-- Booking_ID: string (nullable = true)
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_year: integer (nullable = true)
 |-- arrival_month: integer (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- booking_status: string (nullable = true)



In [5]:
stages = []

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder 

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')
    
    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec')
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()
    
    stages += [string_indexer, encoder]

StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [7]:
stages += [StringIndexer(inputCol='booking_status', outputCol= 'booking_status_index')]

In [8]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [9]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time', 'arrival_year', 'arrival_date',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [10]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'arrival_year',
 'arrival_date',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [11]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

stages += [assembler]

In [12]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  29040
Test size:  7235


In [13]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='features', labelCol='booking_status_index', maxIter=10)

stages.append(gbt)

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=stages)

In [16]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.stepSize, [0.1, 0.2]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol='booking_status_index', predictionCol='prediction', metricName='accuracy')

cross_validator = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

best_model = cross_validator.fit(train_data)

24/10/12 22:29:20 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/12 22:29:20 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1018.4 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1013.3 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1013.8 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1014.4 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1015.6 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1017.9 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1022.4 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1030.8 KiB
24/10/12 22:29:43 WARN DAGScheduler: Broadcasting large task binary wit

In [17]:
predictions = best_model.transform(test_data)

In [19]:
accuracy = evaluator.evaluate(predictions)
print(f'Test Accuracy = {accuracy}')

Test Accuracy = 0.8837595024187975


24/10/12 22:32:27 WARN DAGScheduler: Broadcasting large task binary with size 1993.1 KiB


In [21]:
dir(best_model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_copyValues',
 '_copy_params',
 '_defaultParamMap',
 '_dummy',
 '_from_java',
 '_from_java_impl',
 '_paramMap',
 '_params',
 '_randomUID',
 '_resetUid',
 '_resolveParam',
 '_set',
 '_setDefault',
 '_shouldOwn',
 '_testOwnParam',
 '_to_java',
 '_to_java_impl',
 '_transform',
 'avgMetrics',
 'bestModel',
 'clear',
 'copy',
 'estimator',
 'estimatorParamMaps',
 'evaluator',
 'explainParam',
 'explainParams',
 'extractParamMap',
 'foldCol',
 'getEstimator',
 'getEstimatorParamMaps',
 'getEval

In [44]:
dir(best_model)
best_model.params
best_model.maxDepth

AttributeError: 'CrossValidatorModel' object has no attribute 'maxDepth'

In [47]:
for metric_value, param_values in zip(best_model.avgMetrics, paramGrid):
    for param, value in param_values.inter
    print(pg)

0.8464236141257763
{Param(parent='GBTClassifier_fd9cdbaa602b', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5, Param(parent='GBTClassifier_fd9cdbaa602b', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='GBTClassifier_fd9cdbaa602b', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.'): 0.1}
0.8535220826081922
{Param(parent='GBTClassifier_fd9cdbaa602b', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5, Param(parent='GBTClassifier_fd9cdbaa602b', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='GBTClassifier_fd9cdbaa602b', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each e